In [1]:
#Dice
#COMPAS for bail decision
#Adult-Income for income prediction
#German-Credit for assessing credit risk
#Dataset from Lending Club for loan decisions: https://www.lendingclub.com/info/download-data.action

#Alibi

#AIX360

## Importing libraries

In [2]:
import time
import pickle
import dice_ml
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf

from sklearn.impute import *
from sklearn.ensemble import *
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, MaxAbsScaler, LabelEncoder

from imblearn.over_sampling import SMOTENC

from xmoai.problems.objectives import *
from xmoai.problems.restrictions import *
from xmoai.setup.configure import generate_counterfactuals_classification_proba

from alibi.explainers import *

pd.set_option('display.max_columns', None)

In [3]:
tf.compat.v1.disable_eager_execution()
tf.keras.backend.clear_session()
tf.compat.v1.keras.backend.get_session().list_devices()

[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, -8977809851569685367),
 _DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 9396617216, 8186646063474593478)]

# Adult Dataset

In [23]:
df_adult = pd.read_csv('../FairMood/datasets/adult_nan.csv').dropna()
df_adult

,age,workclass,fnlwgt,education,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
1,82,Private,132870,HS-grad,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,0
3,54,Private,140359,7th-8th,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,0
4,41,Private,264663,Some-college,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,0
5,34,Private,216864,HS-grad,Divorced,Other-service,Unmarried,White,Female,0,3770,45,United-States,0
6,38,Private,150601,10th,Separated,Adm-clerical,Unmarried,White,Male,0,3770,40,United-States,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,22,Private,310152,Some-college,Never-married,Protective-serv,Not-in-family,White,Male,0,0,40,United-States,0
32557,27,Private,257302,Assoc-acdm,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,0
32558,40,Private,154374,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,1
32559,58,Private,151910,HS-grad,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,0


## Defining required columns

In [5]:
num_indexes = []
cat_indexes = []

num_columns = ['age', 'fnlwgt', 'capital.gain', 'capital.loss', 'hours.per.week']
cat_columns = ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country']
integer_columns = num_columns
target = 'income'

for i in range(df_adult.shape[1]):
    col = df_adult.columns[i]
    if col in num_columns:
        num_indexes.append(i)
    elif col in cat_columns:
        cat_indexes.append(i)

### Converting string-encoded categories to integers

In [6]:
label_encoders = {}

for col in cat_columns:
    encoder = LabelEncoder().fit(df_adult[col])
    label_encoders[col] = encoder
    df_adult[col] = encoder.transform(df_adult[col])

## Training

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df_adult.drop(target, axis=1), df_adult[target], test_size=0.7, random_state=0)
X_train, y_train = SMOTENC(categorical_features=np.isin(df_adult.columns, cat_columns)).fit_resample(X_train, y_train)

In [8]:
one_hot_encode = False

# defining both numeric and categorical transformers
numeric_transformer = Pipeline(
    steps=[("imputer", KNNImputer()), ("scaler", MaxAbsScaler())]
)

# setting-up the preprocessing steps
if one_hot_encode:
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse=False)
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, num_indexes),
            ("cat", categorical_transformer, cat_indexes),
        ]
    )
else:
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, num_indexes + cat_indexes),
        ]
    )
    
# defining the model pipeline and training
model = Pipeline(
    steps=[("preprocessor", preprocessor),
           ("classifier", RandomForestClassifier(n_jobs=-1))]
).fit(X_train, y_train)

## Preparing counterfactual generation with DiCE

In [9]:
continuous_precision = {}
for col in num_columns:
    continuous_precision[col] = 0

d = dice_ml.Data(dataframe=pd.concat([X_train, y_train], axis=1),
                 continuous_features=num_columns,
                 continuous_features_precision=continuous_precision,
                 outcome_name=target)
m = dice_ml.Model(model=model, backend='sklearn')
exp = dice_ml.Dice(d,m)

## Preparing counterfactual generation with XMOAI

In [10]:
columns = X_train.columns
categorical_columns_one_hot_encoded = []
categorical_columns_label_encoded = {}

if one_hot_encode and type(model)==Pipeline:
    for cat_col in cat_columns:
        columns_in_cat = [col for col in columns if col.startswith(f'{cat_col}_')]
        columns_in_cat = np.argwhere(np.isin(columns, columns_in_cat)).flatten()
        
        if len(columns_in_cat) > 0:
            categorical_columns_one_hot_encoded.append(columns_in_cat)
else:
    for i in range(len(X_train.columns)):
        if X_train.columns[i] in cat_columns:
            categorical_columns_label_encoded[i] = np.sort(X_train[X_train.columns[i]].unique())
        
display(categorical_columns_one_hot_encoded)
display(categorical_columns_label_encoded)

[]

{1: array([0, 1, 2, 3, 4, 5, 6]),
 3: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]),
 4: array([0, 1, 2, 3, 4, 5, 6]),
 5: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),
 6: array([0, 1, 2, 3, 4, 5]),
 7: array([0, 1, 2, 3, 4]),
 8: array([0, 1]),
 12: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
        35, 36, 37, 38, 39, 40])}

In [11]:
# generating counterfactuals
immutable_column_indexes = [] # let's say we can't change the last column
y_acceptable_range = [0.51, 1.0] # we will only accept counterfactuals with the predicted prob. in this range

upper_bounds = np.array(X_train.max(axis=0)*1.0) # this is the maximum allowed number per column
lower_bounds = np.array(X_train.min(axis=0)*1.0) # this is the minimum allowed number per column.
# you may change the bounds depending on the needs specific to the individual being trained.

## Generating counterfactuals

In [12]:
cat_vars_ord = {}
for i in categorical_columns_label_encoded.keys():
    cat_vars_ord[i] = len(np.unique(X_train.values[:, i]))
cat_vars_ord

cf = CounterfactualProto(sess=tf.compat.v1.keras.backend.get_session(),
                         predict=lambda x: model.predict_proba(x),
                         shape=(1,) + X_test.shape[1:],
                         cat_vars=cat_vars_ord,
                         feature_range=(np.array([X_train.min().values]).astype(np.float32), np.array([X_train.max().values]).astype(np.float32)),
                         max_iterations=200,
                         c_steps=3
                        )
cf.fit(X_train.values)

CounterfactualProto(meta={
  'name': 'CounterfactualProto',
  'type': ['blackbox', 'tensorflow', 'keras'],
  'explanations': ['local'],
  'params': {
              'kappa': 0.0,
              'beta': 0.1,
              'feature_range': (array([[1.7000e+01, 0.0000e+00, 1.3769e+04, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 1.0000e+00, 0.0000e+00]], dtype=float32), array([[9.000000e+01, 6.000000e+00, 1.455435e+06, 1.500000e+01,
        6.000000e+00, 1.300000e+01, 5.000000e+00, 4.000000e+00,
        1.000000e+00, 9.999900e+04, 2.824000e+03, 9.900000e+01,
        4.000000e+01]], dtype=float32)),
              'gamma': 0.0,
              'theta': 0.0,
              'cat_vars': {1: 7, 3: 16, 4: 7, 5: 14, 6: 6, 7: 5, 8: 2, 12: 40},
              'ohe': False,
              'use_kdtree': False,
              'learning_rate_init': 0.01,
              'max_iterations': 200,
              'c_init': 10.0,
              'c_steps':

In [13]:
number_of_instances = 150
dice_all_exp = exp.generate_counterfactuals(X_test.iloc[:number_of_instances], total_CFs=100, desired_class="opposite")

  1%|▊                                                                                                                      | 1/150 [00:13<34:44, 13.99s/it]

Only 45 (required 100)  Diverse Counterfactuals found for the given configuration, perhaps try with different parameters... ; total time taken: 00 min 13 sec


 42%|███████████████████████████████████████████████▉                                                                  | 63/150 [1:43:37<2:08:46, 88.81s/it]

Only 57 (required 100)  Diverse Counterfactuals found for the given configuration, perhaps try with different parameters... ; total time taken: 00 min 16 sec


 56%|████████████████████████████████████████████████████████████████▉                                                   | 84/150 [1:51:56<24:01, 21.84s/it]

...

 62%|███████████████████████████████████████████████████████████████████████▉                                            | 93/150 [1:58:01<38:14, 40.25s/it]

Only 97 (required 100)  Diverse Counterfactuals found for the given configuration, perhaps try with different parameters... ; total time taken: 00 min 27 sec


 85%|██████████████████████████████████████████████████████████████████████████████████████████████▊                 | 127/150 [2:56:35<3:16:53, 513.61s/it]

Only 96 (required 100)  Diverse Counterfactuals found for the given configuration, perhaps try with different parameters... ; total time taken: 27 min 35 sec


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150/150 [3:17:31<00:00, 79.01s/it]


In [ ]:
results = {}

In [25]:
tn, fp, fn, tp = confusion_matrix(y_test.iloc[:number_of_instances], model.predict(X_test.iloc[:number_of_instances])).ravel()
display((tn, fp, fn, tp))

display(y_test[(y_test==1) & (y_test==model.predict(X_test))].index[:5])
display(y_test[(y_test==0) & (y_test==model.predict(X_test))].index[:5])

(98, 10, 12, 30)

Int64Index([2086, 3095, 24220, 25539, 14862], dtype='int64')

Int64Index([29045, 27504, 9264, 21565, 28767], dtype='int64')

In [16]:
# individual to be evaluated
def calculate_metrics(df, X_current, y_desired, method, categorical_columns_label_encoded,
                      categorical_columns_one_hot_encoded):
    f1, prediction = get_difference_target_classification_proba(model, df, y_desired, method)
    f2 = get_difference_attributes(df.values, X_current.values, ranges,
                                   categorical_columns_label_encoded,
                                   categorical_columns_one_hot_encoded)
    f3 = get_modified_attributes(df, X_current, categorical_columns_one_hot_encoded)
    
    return f1, f2, f3, prediction

ranges = (X_train.max() - X_train.min()).values

for index_to_refer in range(number_of_instances):
    print(f'Processing {index_to_refer}')
    X_current = X_test.iloc[index_to_refer:index_to_refer+1]
    y_desired = int(not bool(np.argmax(model.predict_proba(X_current))))

    # DiCE CFs
    print('Processing DiCE')
    try:
        df_dice = dice_all_exp.cf_examples_list[index_to_refer].final_cfs_df.copy().drop(target, axis=1)

        f1, f2, f3, prediction = calculate_metrics(df_dice, X_current, y_desired, 'predict_proba',
                                                   categorical_columns_label_encoded,
                                                   categorical_columns_one_hot_encoded)

        for col in label_encoders.keys():
            df_dice[col] = label_encoders[col].inverse_transform(df_dice[col].astype(int))

        df_dice = pd.concat([df_dice, pd.DataFrame(np.vstack([f1, f2, f3]).T, columns=['F1', 'F2', 'F3'])], axis=1)
        df_dice['Algorithm'] = 'DiCE'
    except:
        df_dice = pd.DataFrame()
        display('Error in DiCE. Resuming...')
    
    # XMOAI CFs
    print('Processing XMOAI')
    try:
        front, X_generated, algorithms = generate_counterfactuals_classification_proba(model, X_train,
                                  X_current.iloc[0], y_desired, immutable_column_indexes,
                                  y_acceptable_range, upper_bounds, lower_bounds,
                                  categorical_columns_label_encoded, categorical_columns_one_hot_encoded,
                                  num_indexes, n_gen=100, pop_size=100, max_changed_vars=5,
                                  verbose=False, select_best=True, seed=0)

        df_xmoai = pd.DataFrame(X_generated.copy(), columns=X_test.columns)
        for col in label_encoders.keys():
            df_xmoai[col] = label_encoders[col].inverse_transform(df_xmoai[col].astype(int))

        df_xmoai = pd.concat([df_xmoai, pd.DataFrame(front, columns=['F1', 'F2', 'F3'])], axis=1)
        df_xmoai['Algorithm'] = 'Proposal'
    except:
        df_xmoai = pd.DataFrame()
        display('Error in XMOAI. Resuming...')

    # Alibi CFs
    print('Processing Alibi')
    try:
        alibi_cf = cf.explain(X_current.values, verbose=False)

        if len(alibi_cf['all'])==0:
            df_alibi = pd.DataFrame()
        else:
            df_alibi = [tuple(v) for v in alibi_cf['all'].values() if len(v) > 0]
            df_alibi = pd.DataFrame([v[0] for v in np.vstack(df_alibi)])
            df_alibi.columns = X_test.columns

            if len(df_alibi)==0:
                continue

            f1, f2, f3, prediction = calculate_metrics(df_alibi, X_current, y_desired, 'predict_proba',
                                                       categorical_columns_label_encoded,
                                                       categorical_columns_one_hot_encoded)

            df_alibi = pd.concat([df_alibi, pd.DataFrame(np.vstack([f1, f2, f3]).T, columns=['F1', 'F2', 'F3'])], axis=1)
            df_alibi['Algorithm'] = 'Alibi'
    except:
        df_alibi = pd.DataFrame()
        display('Error in Alibi. Resuming...')

    results[X_current.index[0]] = pd.concat([df_dice, df_alibi, df_xmoai]).reset_index(drop=True)

Processing 0
Processing DiCE
Processing XMOAI
Processing Alibi


No counterfactual found!


Processing 1
Processing DiCE
Processing XMOAI
Processing Alibi
Processing 2
Processing DiCE
Processing XMOAI
Processing Alibi


No counterfactual found!


Processing 3
Processing DiCE
Processing XMOAI
Processing Alibi


...

Processing 146
Processing DiCE
Processing XMOAI
Processing Alibi


No counterfactual found!


Processing 147
Processing DiCE
Processing XMOAI
Processing Alibi
Processing 148
Processing DiCE
Processing XMOAI
Processing Alibi


No counterfactual found!


Processing 149
Processing DiCE
Processing XMOAI
Processing Alibi


In [17]:
import pickle
pickle.dump([results, X_test.iloc[:number_of_instances]], open('adult_results.pkl', 'wb'))